# Julia is fast

Very often, benchmarks are used to compare languages.  These benchmarks can lead to long discussions, first as to exactly what is being benchmarked and secondly what explains the differences.  These simple questions can sometimes get more complicated than you at first might imagine.

The purpose of this notebook is for you to see a simple benchmark for yourself.  One can read the notebook and see what happened on the author's Macbook Pro with a 4-core Intel Core I7, or run the notebook yourself.

(This material began life as a wonderful lecture by Steven Johnson at MIT: https://github.com/stevengj/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb.)

# Outline of this notebook

- Define the sum function
- Implementations & benchmarking of sum in...
    - C (hand-written)
    - C (hand-written with -ffast-math)
    - python (built-in)
    - python (numpy)
    - python (hand-written)
    - Julia (built-in)
    - Julia (hand-written)
    - Julia (hand-written with SIMD)
- Summary of benchmarks

# `sum`: An easy enough function to understand

Consider the  **sum** function `sum(a)`, which computes
$$
\mathrm{sum}(a) = \sum_{i=1}^n a_i,
$$
where $n$ is the length of `a`.

In [1]:
a = rand(10^7) # 1D vector of random numbers, uniform on [0,1)

10000000-element Vector{Float64}:
 0.09341663173817782
 0.22181509050355652
 0.1142786962478437
 0.5148922855969917
 0.4961649835345512
 0.871785540240195
 0.39879041185902664
 0.8794314734061615
 0.37075445360335
 0.2284125816690974
 ⋮
 0.8469893333375238
 0.4604705537414314
 0.7543126700396453
 0.6047795621618326
 0.570856607739809
 0.4323490244393957
 0.05170899882478952
 0.08716738641302102
 0.346508886912561

In [2]:
sum(a)

4.998800801508116e6

The expected result is 0.5 * 10^7, since the mean of each entry is 0.5

# Benchmarking a few ways in a few languages

In [3]:
@time sum(a)

  0.001215 seconds (1 allocation: 16 bytes)


4.998800801508116e6

In [4]:
@time sum(a)

  0.001284 seconds (1 allocation: 16 bytes)


4.998800801508116e6

In [5]:
@time sum(a)

  0.001329 seconds (1 allocation: 16 bytes)


4.998800801508116e6

The `@time` macro can yield noisy results, so it's not our best choice for benchmarking!

Luckily, Julia has a `BenchmarkTools.jl` package to make benchmarking easy and accurate:

In [50]:
using Pkg
Pkg.add("BenchmarkTools")

   Resolving package versions...
   Installed BenchmarkTools ─ v1.6.0
    Updating `~/.julia/environments/v1.11/Project.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
    Updating `~/.julia/environments/v1.11/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v1.6.0
  [9abbd945] + Profile v1.11.0
Precompiling project...
    591.1 ms  ✓ BenchmarkTools
  1 dependency successfully precompiled in 1 seconds. 205 already precompiled.


In [51]:
using BenchmarkTools  

#  1. The C language

C is often considered the gold standard: difficult on the human, nice for the machine. Getting within a factor of 2 of C is often satisfying. Nonetheless, even within C, there are many kinds of optimizations possible that a naive C writer may or may not get the advantage of.

The current author does not speak C, so he does not read the cell below, but is happy to know that you can put C code in a Julia session, compile it, and run it. Note that the `"""` wrap a multi-line string.

In [52]:
using Libdl
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file


# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

clang: error: unsupported option '-msse3' for target 'arm64-apple-darwin24.4.0'


ProcessFailedException: failed process: Process(`gcc -fPIC -O3 -msse3 -xc -shared -o /var/folders/g9/1bpwq9dx5pj1ck0tcv_m55580000gn/T/jl_qJINiFkhel.dylib -`, ProcessExited(1)) [1]


In [53]:
c_sum(a)

UndefVarError: UndefVarError: `c_sum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [54]:
c_sum(a) ≈ sum(a) # type \approx and then <TAB> to get the ≈ symbolb

UndefVarError: UndefVarError: `c_sum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [55]:
c_sum(a) - sum(a)  

UndefVarError: UndefVarError: `c_sum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [56]:
≈  # alias for the `isapprox` function

isapprox (generic function with 22 methods)

In [57]:
?isapprox

Base.Meta.ParseError: ParseError:
# Error @ /Users/shufanzhang/Documents/coderepos/JuliaTutorials/introductory-tutorials/intro-to-julia/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X26sZmlsZQ==.jl:1:1
?isapprox
╙ ── not a unary operator

We can now benchmark the C code directly from Julia:

In [58]:
c_bench = @benchmark c_sum($a)

UndefVarError: UndefVarError: `c_sum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [59]:
println("C: Fastest time was $(minimum(c_bench.times) / 1e6) msec")

UndefVarError: UndefVarError: `c_bench` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [60]:
d = Dict()  # a "dictionary", i.e. an associative array
d["C"] = minimum(c_bench.times) / 1e6  # in milliseconds
d

UndefVarError: UndefVarError: `c_bench` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [61]:
using Plots
gr()

Plots.GRBackend()

In [62]:
using Statistics # bring in statistical support for standard deviations
t = c_bench.times / 1e6 # times in milliseconds
m, σ = minimum(t), std(t)

histogram(t, bins=500,
    xlim=(m - 0.01, m + σ),
    xlabel="milliseconds", ylabel="count", label="")

UndefVarError: UndefVarError: `c_bench` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# 2. C with -ffast-math

If we allow C to re-arrange the floating point operations, then it'll vectorize with SIMD (single instruction, multiple data) instructions.

In [63]:
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

clang: error: unsupported option '-msse3' for target 'arm64-apple-darwin24.4.0'


ProcessFailedException: failed process: Process(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o /var/folders/g9/1bpwq9dx5pj1ck0tcv_m55580000gn/T/jl_CjCleVkDIl.dylib -`, ProcessExited(1)) [1]


In [64]:
c_fastmath_bench = @benchmark $c_sum_fastmath($a)

UndefVarError: UndefVarError: `c_sum_fastmath` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [65]:
d["C -ffast-math"] = minimum(c_fastmath_bench.times) / 1e6  # in milliseconds

UndefVarError: UndefVarError: `c_fastmath_bench` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# 3. Python's built in `sum` 

The `PyCall` package provides a Julia interface to Python:

In [66]:
# using Pkg; Pkg.add("PyCall")
using PyCall

ArgumentError: ArgumentError: Package PyCall not found in current path.
- Run `import Pkg; Pkg.add("PyCall")` to install the PyCall package.

In [67]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

UndefVarError: UndefVarError: `pybuiltin` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [68]:
pysum(a)

UndefVarError: UndefVarError: `pysum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [69]:
pysum(a) ≈ sum(a)

UndefVarError: UndefVarError: `pysum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [70]:
py_list_bench = @benchmark $pysum($a)

UndefVarError: UndefVarError: `pysum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [71]:
d["Python built-in"] = minimum(py_list_bench.times) / 1e6
d

UndefVarError: UndefVarError: `py_list_bench` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# 4. Python: `numpy` 

## Takes advantage of hardware "SIMD", but only works when it works.

`numpy` is an optimized C library, callable from Python.
It may be installed within Julia as follows:

In [72]:
# using Pkg; Pkg.add("Conda")
using Conda

ArgumentError: ArgumentError: Package Conda not found in current path.
- Run `import Pkg; Pkg.add("Conda")` to install the Conda package.

In [73]:
# Conda.add("numpy")

In [74]:
numpy_sum = pyimport("numpy")["sum"]

py_numpy_bench = @benchmark $numpy_sum($a)

UndefVarError: UndefVarError: `pyimport` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [75]:
numpy_sum(a)

UndefVarError: UndefVarError: `numpy_sum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [76]:
numpy_sum(a) ≈ sum(a)

UndefVarError: UndefVarError: `numpy_sum` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [77]:
d["Python numpy"] = minimum(py_numpy_bench.times) / 1e6
d

UndefVarError: UndefVarError: `py_numpy_bench` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# 5. Python, hand-written 

In [78]:
py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

LoadError: LoadError: UndefVarError: `@py_str` not defined in `Main`
Suggestion: check for spelling errors or missing imports.
in expression starting at /Users/shufanzhang/Documents/coderepos/JuliaTutorials/introductory-tutorials/intro-to-julia/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X65sZmlsZQ==.jl:1

In [79]:
py_hand = @benchmark $sum_py($a)

UndefVarError: UndefVarError: `sum_py` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [80]:
sum_py(a)

UndefVarError: UndefVarError: `sum_py` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [81]:
sum_py(a) ≈ sum(a)

UndefVarError: UndefVarError: `sum_py` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [82]:
d["Python hand-written"] = minimum(py_hand.times) / 1e6
d

UndefVarError: UndefVarError: `py_hand` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# 6. Julia (built-in) 

## Written directly in Julia, not in C!

In [83]:
@which sum(a)

sum(a::AbstractArray; dims, kw...)
     @ Base reducedim.jl:982

In [84]:
j_bench = @benchmark sum($a)

BenchmarkTools.Trial: 4094 samples with 1 evaluation per sample.
 Range (min … max):  1.092 ms …   5.501 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.156 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.216 ms ± 192.561 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂▃▆█▇▆▅▅▄▄▃▄▂▂▁▁▁▁▁▁▁▁   ▁                                  ▁
  ████████████████████████████▇█▇█▇▇▇▇█▆▆▆▇▆▆▆▆▆▅▄▅▅▆▄▃▅▅▄▃▅▃ █
  1.09 ms      Histogram: log(frequency) by time      1.79 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [85]:
d["Julia built-in"] = minimum(j_bench.times) / 1e6
d

Dict{Any, Any} with 1 entry:
  "Julia built-in" => 1.09229

# 7. Julia (hand-written) 

In [86]:
function mysum(A)   
    s = 0.0 # s = zero(eltype(a))
    for a in A
        s += a
    end
    s
end

mysum (generic function with 1 method)

In [87]:
j_bench_hand = @benchmark mysum($a)

BenchmarkTools.Trial: 985 samples with 1 evaluation per sample.
 Range (min … max):  4.906 ms …  5.673 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.066 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.078 ms ± 83.188 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

           ▁▁▅▄▆▆▁█▃                                          
  ▂▁▂▁▃▃▃▅▆█████████▇▆▆▅▄▄▃▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▁▂▁▁▂▂▁▁▂▁▂ ▃
  4.91 ms        Histogram: frequency by time        5.55 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [88]:
d["Julia hand-written"] = minimum(j_bench_hand.times) / 1e6
d

Dict{Any, Any} with 2 entries:
  "Julia hand-written" => 4.90646
  "Julia built-in"     => 1.09229

# 8. Julia (hand-written w. simd) 

In [89]:
function mysum_simd(A)   
    s = 0.0 # s = zero(eltype(A))
    @simd for a in A
        s += a
    end
    s
end

mysum_simd (generic function with 1 method)

In [90]:
j_bench_hand_simd = @benchmark mysum_simd($a)

BenchmarkTools.Trial: 4108 samples with 1 evaluation per sample.
 Range (min … max):  1.091 ms …   5.283 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.145 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.213 ms ± 204.749 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂▅▇█▇▅▅▄▃▃▃▃▃▃▂▂▂▁▂▁▂▁▁▁                                    ▂
  ████████████████████████████▇████▇████▇█▆▇▆█▆▆▆▆▆▅▅▆▅▆▃▆▁▅▆ █
  1.09 ms      Histogram: log(frequency) by time       1.8 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [91]:
mysum_simd(a)

4.998800801508154e6

In [92]:
d["Julia hand-written simd"] = minimum(j_bench_hand_simd.times) / 1e6
d

Dict{Any, Any} with 3 entries:
  "Julia hand-written simd" => 1.09096
  "Julia hand-written"      => 4.90646
  "Julia built-in"          => 1.09229

# Summary

In [93]:
for (key, value) in sort(collect(d), by=last)
    println(rpad(key, 25, "."), lpad(round(value; digits=1), 6, "."))
end

Julia hand-written simd.....1.1
Julia built-in..............1.1
Julia hand-written..........4.9
